In [1]:
import numpy as np
import re
import pandas as pd
from collections import Counter 

'''
    提取句子关键词
    这里尚未对未录入词库的词进行处理
'''


maxwords = 100  #最大关键词数
sentences_num = 150
traindata_path = './data_processed/train_data.csv'
all_data = pd.read_csv(traindata_path, header=None, engine="python", encoding="utf-8")  #8597
all_data.columns = ["index","words","types","caijing","fangchan","jiaoyu","junshi","keji","qiche","tiyu","youxi","yule"]
all_data.drop(0,inplace=True)
all_data.drop(['index',"caijing","fangchan","jiaoyu","junshi","keji","qiche","tiyu","youxi","yule"],inplace=True,axis=1)


print(' ===============================  data loading success! ===================================')

 ===============================  data loading success! ===================================


In [2]:
all_data

,words,types
1,北京 王恩博 中国 金融业 开放 程度 不断 加深 越来越 外资 机构 来华 展业 中国 银...,caijing
2,北京 王恩博 中国 银 保监会 获悉 官方 坚决 惩治 重大 金融风险 背后 腐败问题 破坏...,caijing
3,天津 张道 正 京津冀 交通 一体化 重点项目 新进展 凌晨时分 最后 一车 混凝土 浇筑 ...,caijing
4,新 经纬 客户端 周三 A股 三大 指数 小幅 低开 能源 设备 家用电器 汽车 造纸 板块...,caijing
5,华盛顿 沙晗 汀 国际货币基金组织 IMF 总裁 格奥尔基 耶娃 当地 时间 表示 IMF ...,caijing
6,长沙 付敬懿 总部 位于 湖南 长沙 中国 工程机械 龙头企业 中联重科 晚间 发布 年度报...,caijing
7,机车 汽笛 一声 长鸣 一列 中欧 班列 江西 南昌 向塘 国际 陆港 新城 横岗 站 出发...,caijing
8,中国 进口 增强 世界 经济 动能 新知 新觉 进口 联通 国内 国际 两个 市场 两种 资...,caijing
9,春季 出游 热潮 到来 预计 今年 清明节 出游 人次 达到 亿人次 继续 做好 疫情 防控...,caijing
10,翻开 中国 地图 北部湾 西南地区 最 便捷 出海口 清晰可见 北海 钦州 防城港 座 港口...,caijing


In [3]:
"""
    加载关键词词典
"""

def get_corpus_dict(sentences_num):
    corpus_dict_list = []
    for i in range(9):
        index_start = sentences_num * i + 1
        index_end = sentences_num * (i+1) + 1
        texts = ""
        for j in range(index_start,index_end):
            texts = texts + all_data['words'][j].strip() + " "
        corpus_dict_list.append(count_words_in_sentence(texts))
    return corpus_dict_list

#定义TF-IDF的计算过程
def count_words_in_sentence(string):
    word_list = string.strip().split(' ')
    count = Counter(word_list)
    dict1 = dict(count)
    return dict1

def get_all_sentence_dict():
    dict_list = []
    for i in range(1,all_data.shape[0]+1):
        dict_list.append(count_words_in_sentence(all_data['words'][i]))
    return dict_list

def get_total_words_num(corpus_dict_list):
    words_num = 0
    for i in range(9):
        words_num += len(corpus_dict_list[i])
    return words_num



In [4]:
dict_list = get_all_sentence_dict() #存储每一个句子词典的列表，里面计算了词频
corpus_dict_list = get_corpus_dict(sentences_num) #存储每一个语料库词典的列表，计算了词频
words_num = get_total_words_num(corpus_dict_list) #存储全语料库中词数

In [5]:
"""
    分词+去停用词
"""

import jieba
import re
stopwords_path = './data/stop_words.txt'
stopwords = [line.strip() for line in open(stopwords_path,encoding='UTF-8').readlines()]

#处理标点、数字
def punct_handling(inputstr):
    remove_str = "[\s+\.:\!-\/_,$%\[\]^\)*(+\"\']+|[+——！1234567890·，。？?、~@#￥%……&*（）` | \" ~〜 “ １　２　３　４　５　６　７　８　９　０”【】 ※]+"
    punc = "！？｡＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."
    punc = punc.encode('utf-8').decode("utf-8")
    string1 = re.sub(remove_str,"",inputstr)
    string1 = re.sub(punc,"",string1)
    return string1


# 对句子进行中文分词
def seg_depart(sentence):
    # 对文档中的每一行进行中文分词
    sentence = punct_handling(sentence)
    sentence_depart = jieba.cut(sentence.strip())
    # 输出结果为outstr
    outstr = ''
    # 去停用词
    for word in sentence_depart:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr

In [26]:
import math

"""
    计算tfidf，转化关键词序列
"""

# tf1 = 某词出现当前句子中的次数 / 全句词数       
# tf2 = 某词出现在该语料库中的次数 / 全语料库词数   
# idf = log(句子总数 / 包含该词的句子数 +1 )

def tf1(word,string):
    word_list = string.strip().split(' ')
    count = Counter(word_list)
    dict1 = dict(count)
    for key in dict1:
        dict1[key] /= len(dict1)
    return dict1[word]

#types = [0,1,2,3,4,5,6,7,8] 分别对应九个类
def tf2_pred(word,corpus_dict_list,words_num):
    max_frequency = 1;
    for i in range(9):
        frequency = corpus_dict_list[i].get(word,-1)
        max_frequency = max(max_frequency,frequency)
    return max_frequency / words_num

def stem_count(word,dict_list):
    count = 0
    for i in range(len(dict_list)):
        if word in dict_list[i]:
            count += 1
    if(count == 0):
        return 1
    return count

def idf(word, dict_list): 
    return math.log((all_data.shape[0]) / (1 + stem_count(word,dict_list)))

def tfidf(word, string, dict_list,corpus_dict_list,words_num):
    return tf1(word, string) * tf2_pred(word,corpus_dict_list,words_num) * idf(word, dict_list)


def get_keyword_sentence_pred(input_str):
    def by_score(t):
        return t[1]
    word_list = input_str.strip().split(' ')
    if(len(word_list) <= maxwords):
        return input_str    #词数过少直接返回
    tfidf_list = []
    for word in word_list:
        tfidf_list.append(tfidf(word,input_str,dict_list,corpus_dict_list,words_num))
    keyword_scores = list(zip(word_list,tfidf_list))   #关键词得分
    keyword_scores = list(set(keyword_scores)) #去重
    
    l1 = sorted(keyword_scores,key=by_score,reverse=True)  #关键词排序
    keyword_list = l1[:maxwords]
    processed_sentence = ""
    for word in keyword_list:
        processed_sentence = processed_sentence + word[0] + ' '
    return processed_sentence.strip()

def process_sentence(input_str):
    splited_str = seg_depart(input_str)
    keyword_sentences = get_keyword_sentence_pred(splited_str)
    return keyword_sentences


In [27]:
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model





In [28]:
import nltk.data
tokenizer = nltk.data.load('./model/textcnn+tfidf.pickle')


In [29]:


#model = load_model('./model/textCNN_TFIDF2_word.h5')


def predict(input_str):
    sentences = process_sentence(input_str)
    print(sentences)
    word_index = tokenizer.word_index
    x_pred_ids = tokenizer.texts_to_sequences([sentences])
    x_pred_padded_seqs=pad_sequences(x_pred_ids,maxwords)
    
    typelist = model.predict(x_pred_padded_seqs)
    print(typelist)
    print(np.argmax(typelist))
    return np.argmax(typelist)




In [34]:
input_str = '标签： 皇马 本泽马 莫德里奇 北京时间3月17日凌晨4时，欧冠1/8决赛次回合，皇马主场3-1战胜亚特兰大，皇马以两回合总比分4-1成功晋级欧冠八强。第34分钟本泽马射门得分，第59分钟拉莫斯罚进点球，第83分钟穆里尔主罚任意球破门，第84分钟阿森西奥进球。欧冠1/8决赛：皇马3-1亚特兰大 总比分4-1 图集已浏览完毕 重新浏览 评论 ( 0 ) 分享： 易信( 0 ) LOFTER 新浪微博 人人网 QQ空间 推荐内容 欧冠1/8决赛：巴黎1-1巴… 欧冠1/8决赛：尤文3-2波… 欧冠：皇马1-0亚特兰大 欧冠：切尔西1-0马竞 欧冠：尤文1-2波尔图 欧冠：巴萨1-4巴黎 欧冠：国米0-0矿工 欧冠：尤文3-0巴萨 回顾梅西C罗欧冠5次交手 传… 欧冠：曼联1-3巴黎 欧冠：皇马0-2矿工 刘翔110米栏仅获铜牌 推荐视频 糖醋里脊没有人不爱吧！酸甜口肉菜… 蒲公英好吃又简单的做法，加入一碗… 上一图片 下一图集 欧冠1/8决赛：巴黎1-1巴…'

In [30]:
#input_str = '广州1月25日电 题：广州南沙集中签约24项目 重点发展智慧城市和新兴产业总投资额约700亿元(不含意向融资及专项贷款额)的24个项目，25日在广州市南沙区举办的“畅想未来、共创湾区‘新’发展—南沙重点产业项目集中签约”活动上进行了集中签约，达产产值/营收合计超1400亿元，涵盖粤港澳合作、智慧城市建设以及战略性新兴产业、金融助力湾区建设等领域。南沙区政府相关负责人表示，本次集中签约活动展现了南沙推进粤港澳全面合作示范区建设取得的成效，也体现了南沙区奋力实现“十四五”开好局起好步，推动“三区一中心”建设迈上新台阶的决心。南沙区委副书记、区长董可介绍，2020年，该区全年新引进世界500强企业投资项目25个、累计197个，集中签约动工重点项目193个、总投资额近4500亿元，新设企业4.9万家，204个重点项目完成年度投资额943.37亿元，超额完成年度总投资计划目标。董可表示，这些项目的落地启动，将为南沙战略性新兴产业和未来产业发展提供强引擎，为南沙“十四五”开局注入新的强劲动力，对提升区域产业竞争力和创新驱动力具有十分重要的意义。本次签约除了港澳特色凸显外，智慧城市建设及数字经济、战略性新兴产业、特色金融等项目也是亮点纷呈。在智慧城市领域，南沙与佳都新太科技股份有限公司签署了共同推进南沙新型智慧城市建设的战略合作框架协议，双方将在交通、公安、应急、住建、车路协同、智能网联汽车、智慧地铁、智慧社区、政府服务等方面开展合作，共同建设智能交通国家新一代人工智能开放创新平台、南沙城市大脑中枢平台和南沙交通大脑研发基地。同时，本次签约还有7个战略性新兴产业。具体项目有中创医疗器械与生物科技产业园、纳诺新能源新材料创业园、瑞高复合新材料创新生态产业园、泽亨高端涂装设备智造总部、光铭精密组件增材智造总部等。此外，在金融助力湾区建设方面，越秀金控与南沙区签署了合作备忘录，目前已完成旗下越秀金控资本50亿元实缴，并将深化基金方面的合作，推进与南沙合作设立专项产业基金；广州银行与南沙区签署战略合作框架协议，在未来3年提供等值人民币300亿元的意向性融资支持，围绕重点项目建设、民生建设发展、特色产业发展、乡村振兴等领域为南沙区提供给更好的金融服务；中国建设银行广州分行与南沙区签署全面战略合作框架协议暨气候投融资合作协议，拟新增500亿贷款规模，支持南沙区基础设施建设、新基建建设、城市更新改造。(完)'

In [31]:
#predict(input_str)

In [45]:
inputs ='原标题：锦织圭第二次新冠检测仍为阳性 能否参加美网成未知8月22日电 北京时间22日凌晨，日本网球名将锦织圭在社交平台上公布了自己的第二次核酸检测结果。不幸的是，锦织圭的检测结果仍旧是阳性。锦织圭表示自己症状很轻，并且处于完全隔离中。在下周初还会做一次核酸检测，届时会向大家公布最新情况。同时，他也感谢了大家对他的关心。五天前，锦织圭宣布自己新冠病毒检测呈阳性，并因此退出即将开始的辛辛那提大师赛。据悉，按美国公共卫生当局规定，新冠阳性需要隔离10天，锦织圭有可能在美网开赛前5天解除隔离。尽管宣布退出了辛辛那提站比赛，但锦织圭却并没有对是否参加美网明确表态。按照计划，2020年美国网球公开赛将于将于8月31日开战。(完)返回搜狐，查看更多<!-- 政务账号添加来源标示处理 --><!-- 政务账号添加来源标示处理 -->'

In [46]:
predict(inputs)

美国 圭 网球 美网 比赛 辛辛那提 检测 锦织 新冠 隔离 参加 阳性 搜狐 公开赛 名将 公布 北京 退出 宣布 返回 查看 却 做 平台 需要 日本 大师赛 时间 规定 计划 表示 情况 完 一次 下 明确 站 社交 添加 处理 开赛 关心 完全 处于 账号 是否 凌晨 第二次 核酸 最新 开战 感谢 能否 来源 政务 呈 不幸 还会 解除 症状 未知 表态 轻 病毒检测 五天 标示 前锦织 仍为 美网成 天锦织 但锦织 周初 公共卫生 当局 前天
[[5.8418841e-06 4.5065266e-05 1.1990718e-06 8.2284994e-03 5.1741739e-07
  3.9020607e-03 9.6149784e-01 2.6210921e-02 1.0814988e-04]]
6


6